<a href="https://colab.research.google.com/github/Ankur099IIT/Natural-Language-Processing-Course-3-Deeplearning.ai/blob/main/Training_a_binary_classifier_with_the_IMDB_Reviews_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
print(tf.__version__)

2.8.0


In [4]:
import tensorflow_datasets as tfds

imdb, info = tfds.load('imdb_reviews', with_info = True, as_supervised = True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteLTUBCE/imdb_reviews-train.tfrecord


  0%|          | 0/25000 [00:00<?, ? examples/s]

...

  0%|          | 0/25000 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteLTUBCE/imdb_reviews-unsupervised.tfrecord


  0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [5]:
import numpy as np

train_data, test_data = imdb['train'], imdb['test']

In [11]:
training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

for s,l in train_data:
  training_sentences.append(str(s.numpy()))
  training_labels.append(l.numpy())

for s,l in test_data:
  testing_sentences.append(str(s.numpy()))
  testing_labels.append(l.numpy())

In [12]:
training_sentences[0:10]

['b"This was an absolutely terrible movie. Don\'t be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie\'s ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor\'s like Christopher Walken\'s good name. I could barely sit through it."',
 "b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot devel

In [13]:
len(training_sentences)

25000

In [14]:
training_labels[:5]

[0, 0, 0, 1, 1]

In [30]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type = 'post'
oov_tok = '<OOV>'

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_tok)

tokenizer.fit_on_texts(training_sentences)

word_index = tokenizer.word_index

sequences = tokenizer.texts_to_sequences(training_sentences)

padded = pad_sequences(sequences, maxlen = max_length, truncating = trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen = max_length)


In [31]:
print(padded[0])

[   0    0   59   12   14   35  439  400   18  174   29    1    9   33
 1378 3401   42  496    1  197   25   88  156   19   12  211  340   29
   70  248  213    9  486   62   70   88  116   99   24 5740   12 3317
  657  777   12   18    7   35  406 8228  178 2477  426    2   92 1253
  140   72  149   55    2    1 7525   72  229   70 2962   16    1 2880
    1    1 1506 4998    3   40 3947  119 1608   17 3401   14  163   19
    4 1253  927 7986    9    4   18   13   14 4200    5  102  148 1237
   11  240  692   13   44   25  101   39   12 7232    1   39 1378    1
   52  409   11   99 1214  874  145   10]


In [33]:
print(testing_padded[0])

[  11 1450 1528   12  251  235   11  216    2  377 6429    3   62   95
   11  174  105   11 1528  180   12  251   37    6 1144    1  682    7
 4452    1    4    1  334    7   37 8367  377    5 1420    1   13   30
   64   28    6  874  181   17    4 1050    5   12  224    3   83    4
  353   33  353 5229    5   10    6 1340 1160    2 5738    1    3    1
    5   10  175  328    7 1319 3989    4  798 1946    5    4  250 2710
  158    3    2  361   31  187   25 1170  499  610    5    2  122    2
  356 1398 7725   30    1  881   38    4   20   39   12    1    4    1
  334    7    4   20  634   60   48  214]


In [34]:
model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length = max_length),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(6, activation = 'relu'),
        tf.keras.layers.Dense(1, activation = 'sigmoid')

])

In [35]:
model.compile(loss = 'binary_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])

In [37]:
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [39]:
model.fit(padded, training_labels_final, epochs = 10, validation_data = (testing_padded, testing_labels_final))

Epoch 1/10
782/782 [==============================] - 6s 6ms/step - loss: 0.5000 - accuracy: 0.7359 - val_loss: 0.3489 - val_accuracy: 0.8484
Epoch 2/10
782/782 [==============================] - 5s 6ms/step - loss: 0.2464 - accuracy: 0.9039 - val_loss: 0.3617 - val_accuracy: 0.8438
Epoch 3/10
782/782 [==============================] - 5s 6ms/step - loss: 0.1092 - accuracy: 0.9708 - val_loss: 0.4432 - val_accuracy: 0.8276
Epoch 4/10
782/782 [==============================] - 5s 6ms/step - loss: 0.0304 - accuracy: 0.9956 - val_loss: 0.5255 - val_accuracy: 0.8229
Epoch 5/10
782/782 [==============================] - 5s 6ms/step - loss: 0.0074 - accuracy: 0.9995 - val_loss: 0.5933 - val_accuracy: 0.8232
Epoch 6/10
782/782 [==============================] - 4s 6ms/step - loss: 0.0023 - accuracy: 1.0000 - val_loss: 0.6455 - val_accuracy: 0.8266
Epoch 7/10
782/782 [==============================] - 5s 6ms/step - loss: 9.6905e-04 - accuracy: 1.0000 - val_loss: 0.6891 - val_accuracy: 0.8272
Ep

In [41]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(10000, 16)


In [43]:
reverse_word_index = tokenizer.index_word

In [44]:
import io

out_v = io.open('vecs.tsv', 'w', encoding = 'utf-8')
out_m = io.open('meta.tsv', 'w', encoding = 'utf-8')

for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]

  out_m.write(word + '\n')
  out_v.write('\t'.join([str(x) for x in embeddings]) + '\n')

out_v.close()
out_m.close()

In [46]:
# To download the files and use in the embedding tensorflow projector

try:
  from google.colab import files

except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>